Front Lens database work

In [21]:
import numpy as np
import pandas as pd
import re


In [22]:
pa_raw = "C:\\SULI2024\\SIMION_files_2024\\TOF_obj_17_high_potential.patxt"
fly_raw = "C:\\SULI2024\\SIMION_files_2024\\300eV (origin 20).fly2"

Potential array

In [14]:
# Dataframe with just field information
pa_df = pd.read_csv(pa_raw, sep=' ', header = 17, names=['x','y','z','is_electrode','potential'], engine='python')
pa_df.head()

,x,y,z,is_electrode,potential
0,0,0.0,0.0,1.0,0.0
1,1,0.0,0.0,1.0,0.0
2,2,0.0,0.0,1.0,0.0
3,3,0.0,0.0,1.0,0.0
4,4,0.0,0.0,1.0,0.0


In [15]:
# List with header information
with open(pa_raw,'r') as file:
    lines = file.readlines()
    
    is_header = False
    pa_header_lines = []
    
    for line in lines:
        if 'begin_header' in line:
            is_header = True
            continue
        
        if 'end_header' in line:
            is_header = False
            continue
        
        if is_header:
            pa_header_lines.append(line.strip())
            
    print(pa_header_lines)
    print('This is a list of ' + str(len(pa_header_lines)) + ' elements.')
            

['mode -1', 'symmetry cylindrical', 'max_voltage 100000', 'nx 995', 'ny 250', 'nz 1', 'mirror_x 0', 'mirror_y 1', 'mirror_z 0', 'field_type electrostatic', 'ng 100', 'fast_adjustable 0', 'data_format x y z is_electrode potential']
This is a list of 13 elements.


Parse fly file


In [1]:
with open(fly_raw,'r') as file:
    #fly_str = file.read()
    fly_lines = file.readlines()
    counter = 0 # There are 5 groups of particles in this file. Counter should be 5 at the end.
    start_parse = False
    
    data_dict = {} #Empty dictionary. Dictionary is populated for each instance.
    list = [] # Empty list. Dictionaries are dumped into this list each time a new instance is parsed.
    nested = False
    parent_key = ''
    for row in fly_lines:

        if "standard_beam" in row:
            counter += 1
            start_parse = True
            if data_dict:
              list.append(data_dict)
              data_dict = {}  
            continue
        elif start_parse:
            data = row.split(' = ')
            
            if len(data)>= 2:
                key = data[0].strip()
                values = data[1].replace(',\n','').strip()
                
            # Special cases
            if '{' in row:
                nested = True
                parent_key = key
                continue
            if nested:
                key = f'{parent_key}_{key}'
            if '}' in row:
                nested = False
                parent_key = ''
                continue
            
            data_dict[key] = values             
        else:
            start_parse = False
   # Append the last chunk
    if counter == 1+ len(list):
        list.append(data_dict)
        
print(list)
        


NameError: name 'fly_raw' is not defined

In [82]:
fly_df = pd.DataFrame(list)
fly_df.head()

,n,tob,mass,charge,ke,az,el_first,el_step,el_n,cwf,color,position
0,31,0,0.000548579903,-1,300,0,30,-2,31,1,0,"vector(20.001, 0, 0)"
1,31,0,0.000548579903,-1,300,0,30,-2,31,1,2,"vector(20.001, 0.01, 0)"
2,31,0,0.000548579903,-1,300,0,30,-2,31,1,2,"vector(20.001, -0.01, 0)"
3,31,0,0.000548579903,-1,300,0,30,-2,31,1,3,"vector(20.001, 0.02, 0)"
4,31,0,0.000548579903,-1,300,0,30,-2,31,1,3,"vector(20.001, -0.02, 0)"
